In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import glob, os, sys
from pathlib import Path
#import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click

import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages

import pyarrow.parquet
import pyarrow as pa
import pandas as pd
import Outcomes


import multiprocessing
import pathlib
import time

import click

from SEIR import seir, setup, file_paths
from SEIR.utils import config
from SEIR.profile import profile_options
from Outcomes import outcomes

os.chdir('COVIDScenarioPipeline/test/test_inference/')

config_path = 'config.yml'

config.set_file(config_path)

FileNotFoundError: [Errno 2] No such file or directory: 'COVIDScenarioPipeline/test/test_inference/'

In [ ]:

spatial_config = config["spatial_setup"]
spatial_base_path = pathlib.Path(spatial_config["base_path"].get())
scenario = 'None'
deathrate = 'high_death_rate'
stoch_traj_flag= True # Truthy: stochastic simulation, Falsy: determnistic mean of the binomial draws
nsim = 10
interactive = False
write_csv = False
write_parquet = True

index = 1
run_id = '2020.09.14.11:56:34.UTC'
prefix='test_inference/None/high_death_rate/2020.09.14.11:56:34.UTC/global/intermediate/000000001.'
s = setup.Setup(
    setup_name=config["name"].get() + "_" + str(scenario),
    spatial_setup=setup.SpatialSetup(
        setup_name=spatial_config["setup_name"].get(),
        geodata_file=spatial_base_path / spatial_config["geodata"].get(),
        mobility_file=spatial_base_path / spatial_config["mobility"].get(),
        popnodes_key=spatial_config["popnodes"].get(),
        nodenames_key=spatial_config["nodenames"].get()
    ),
    nsim=nsim,
    npi_scenario=scenario,
    npi_config=config["interventions"]["settings"][scenario],
    seeding_config=config["seeding"],
    ti=config["start_date"].as_date(),
    tf=config["end_date"].as_date(),
    interactive=interactive,
    write_csv=write_csv,
    write_parquet=write_parquet,
    dt=config["dt"].as_number(),
    first_sim_index = index,
    in_run_id = run_id,
    in_prefix = prefix,
    out_run_id = run_id,
    out_prefix = prefix
)

print(f"""
>> Running ***{'STOCHASTIC' if stoch_traj_flag else 'DETERMINISTIC'}*** SEIR and Outcomes modules
>> Scenario: {scenario}
>> Starting {s.nsim} model runs beginning from {s.first_sim_index}
>> Setup *** {s.setup_name} *** from {s.ti} to {s.tf}
>> writing to folder : {s.datadir}{s.setup_name}
    """)
setup_name = s.setup_name


onerun_OUTCOMES_loadID = lambda index: outcomes.onerun_delayframe_outcomes_load_hpar(config,
                                                                                     run_id, prefix, int(index), # input
                                                                                     run_id, prefix, int(index), # output
                                                                                     deathrate, stoch_traj_flag)
onerun_OUTCOMES = lambda index: outcomes.run_delayframe_outcomes(config,
                                                                 run_id, prefix, int(index), # input
                                                                 run_id, prefix, int(index), # output
                                                                 deathrate, nsim=1, n_jobs=1, stoch_traj_flag = stoch_traj_flag)
onerun_SEIR_loadID = lambda sim_id2write, s, sim_id2load: seir.onerun_SEIR_loadID(int(sim_id2write), s, int(sim_id2load), stoch_traj_flag)
onerun_SEIR = lambda sim_id2write, s: seir.onerun_SEIR(int(sim_id2write), s, stoch_traj_flag)

onerun_OUTCOMES(0)
#onerun_OUTCOMES_loadID(0)